In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

movies = pd.read_csv('./Boltzmann_Machines/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

users = pd.read_csv('./Boltzmann_Machines/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

ratings = pd.read_csv('./Boltzmann_Machines/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

movies.head()

users.head()

ratings.head()

training_set = pd.read_csv('Boltzmann_Machines/ml-100k/u1.base', delimiter='\t')

training_set.info()

training_set = np.array(training_set, dtype = 'int')

test_set = pd.read_csv('Boltzmann_Machines/ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype = 'int')

training_set[:,0]

nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

# converting the data into Torch tensors
# a tensor is just a multidimensional matrix
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79999 entries, 0 to 79998
Data columns (total 4 columns):
1            79999 non-null int64
1.1          79999 non-null int64
5            79999 non-null int64
874965758    79999 non-null int64
dtypes: int64(4)
memory usage: 2.4 MB


In [4]:
# Creating the autoencoder class and the architecture of the neural network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x)) # encoding
        x = self.activation(self.fc2(x)) # encoding
        x = self.activation(self.fc3(x)) # decoding
        x = self.fc4(x)
        return x

In [5]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)


In [8]:
# Training the stacked autoencoder
nb_epoch = 100
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input =Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 0.9813570944642322
epoch: 2 loss: 0.9805559879645873
epoch: 3 loss: 0.9779638236035247
epoch: 4 loss: 0.975960627912988
epoch: 5 loss: 0.9768910592220851
epoch: 6 loss: 0.971290257482184
epoch: 7 loss: 0.9670972828633658
epoch: 8 loss: 0.9705565866507261
epoch: 9 loss: 0.9668982812829936
epoch: 10 loss: 0.9648471598029071
epoch: 11 loss: 0.96573215149914
epoch: 12 loss: 0.9612606818789257
epoch: 13 loss: 0.9629712332290681
epoch: 14 loss: 0.9596853593656918
epoch: 15 loss: 0.9580204662701146
epoch: 16 loss: 0.9568091285819404
epoch: 17 loss: 0.9574796518419522
epoch: 18 loss: 0.9547364338542567
epoch: 19 loss: 0.9552848512325566
epoch: 20 loss: 0.9525551512333831
epoch: 21 loss: 0.9514403363986381
epoch: 22 loss: 0.9503830045200484
epoch: 23 loss: 0.9507152156653442
epoch: 24 loss: 0.9474885086011631
epoch: 25 loss: 0.9480141384474365
epoch: 26 loss: 0.945792338391285
epoch: 27 loss: 0.9462790661635331
epoch: 28 loss: 0.9441346043455423
epoch: 29 loss: 0.945261091092477


In [10]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input =Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.9503837958788115
